# Extract and Transform - Injury Data
Due to the size and inconsistencies in the data, the data are not ready to be loaded into a database until cleaned and transformed. 
To address the efficiency of processing the high volume data, I will be using polars instead of pandas, which does have some significant differences in syntax. 
Performance metrics collected have shown that polars is saving minutes of processing time between the same operations using SQL or pandas. 

### Joins
Smaller tables will be joined as dataframes. The tracking data is very large, and for efficiency, these tables will be transformed as lazyframes, and only when the columns 
have been transformed and isolated will they be read in as dataframes. 

In [2]:
PlayList_path = "F:/Data/nfl-playing-surface-analytics/PlayList.csv"
InjuryRecord_path = "F:/Data/nfl-playing-surface-analytics/InjuryRecord.csv"
PlayerTrackData_path = "F:/Data/nfl-playing-surface-analytics/PlayerTrackData.csv"

In [3]:
import polars as pl
import numpy as np

In [6]:
plays = pl.read_csv(PlayList_path)
injuries = pl.read_csv(InjuryRecord_path)

In [8]:
plays.head()

PlayerKey,GameID,PlayKey,RosterPosition,PlayerDay,PlayerGame,StadiumType,FieldType,Temperature,Weather,PlayType,PlayerGamePlay,Position,PositionGroup
i64,str,str,str,i64,i64,str,str,i64,str,str,i64,str,str
26624,"""26624-1""","""26624-1-1""","""Quarterback""",1,1,"""Outdoor""","""Synthetic""",63,"""Clear and warm""","""Pass""",1,"""QB""","""QB"""
26624,"""26624-1""","""26624-1-2""","""Quarterback""",1,1,"""Outdoor""","""Synthetic""",63,"""Clear and warm""","""Pass""",2,"""QB""","""QB"""
26624,"""26624-1""","""26624-1-3""","""Quarterback""",1,1,"""Outdoor""","""Synthetic""",63,"""Clear and warm""","""Rush""",3,"""QB""","""QB"""
26624,"""26624-1""","""26624-1-4""","""Quarterback""",1,1,"""Outdoor""","""Synthetic""",63,"""Clear and warm""","""Rush""",4,"""QB""","""QB"""
26624,"""26624-1""","""26624-1-5""","""Quarterback""",1,1,"""Outdoor""","""Synthetic""",63,"""Clear and warm""","""Pass""",5,"""QB""","""QB"""


In [9]:
injuries.head()

PlayerKey,GameID,PlayKey,BodyPart,Surface,DM_M1,DM_M7,DM_M28,DM_M42
i64,str,str,str,str,i64,i64,i64,i64
39873,"""39873-4""","""39873-4-32""","""Knee""","""Synthetic""",1,1,1,1
46074,"""46074-7""","""46074-7-26""","""Knee""","""Natural""",1,1,0,0
36557,"""36557-1""","""36557-1-70""","""Ankle""","""Synthetic""",1,1,1,1
46646,"""46646-3""","""46646-3-30""","""Ankle""","""Natural""",1,0,0,0
43532,"""43532-5""","""43532-5-69""","""Ankle""","""Synthetic""",1,1,1,1


In [1]:
def data_loader(dataset): 
    """
    Accepts the desired dataset string and opens the file as either a polars dataframe
    or a lazyframe. Lazyloading is used for the larger tracking datasets. 
    """
    import polars as pl  # type: ignore
    import numpy as np # type: ignore

    valid_datasets = ['plays', 'injuries', 'role_data', 'punt_data', 'play_information', 'game_data', 'video_review']
    if dataset not in valid_datasets: 
        raise ValueError(f"Invalid dataset name '{dataset}'. Valid options are: {valid_datasets}")

    try:
        # Injury Datasets
        if dataset == 'plays':
            PlayList_path = "F:/Data/nfl-playing-surface-analytics/PlayList.csv"
            df = pl.read_csv(PlayList_path)
        elif dataset == 'injuries':
            InjuryRecord_path = "F:/Data/nfl-playing-surface-analytics/InjuryRecord.csv"
            df = pl.read_csv(InjuryRecord_path)

        # Concussion Datasets
        elif dataset == 'role_data':
            play_player_role_data_path = "F:/Data/NFL-Punt-Analytics-Competition/play_player_role_data.csv"
            df = pl.read_csv(play_player_role_data_path)
        elif dataset == 'punt_data':
            player_punt_data_path = "F:/Data/NFL-Punt-Analytics-Competition/player_punt_data.csv"
            df = pl.read_csv(player_punt_data_path)
        elif dataset == 'play_information':
            play_information_path = "F:/Data/NFL-Punt-Analytics-Competition/play_information.csv"
            df = pl.read_csv(play_information_path)
        elif dataset == 'game_data':
            game_data_path = "F:/Data/NFL-Punt-Analytics-Competition/game_data.csv"
            df = pl.read_csv(game_data_path)
        elif dataset == 'video_review':
            video_review_path = "F:/Data/NFL-Punt-Analytics-Competition/video_review.csv"
            df = pl.read_csv(video_review_path)

        return df
    
    except Exception as e: 
        print(f"An error occurred while loading the dataset '{dataset}': {e}")
        return None

In [2]:
def table_joiner(analysis):
    """
    Joins the plays and injuries tables in the surface injury data 
    """
    import polars as pl # type: ignore
    # from DataHandler import data_loader

    valid_analyses = ['injury', 'concussion']
    if analysis not in valid_analyses: 
        raise ValueError(f"Invalid dataset name '{analysis}'. Valid options are: {valid_analyses}")


    try: 
        #Injury Data Loader
        if analysis == 'injury':
            plays = data_loader('plays')
            injuries = data_loader('injuries')

            df = (
                plays.join(injuries, on="PlayKey", how='left')
                .select([
                    pl.col("PlayKey")
                    , pl.col("Position")
                    , pl.col("StadiumType")
                    , pl.col("FieldType")
                    , pl.col("Temperature")
                    , pl.col("Weather")
                    , pl.col("PlayType")
                    , pl.col("BodyPart")
                    , pl.col("DM_M1")
                    , pl.col("DM_M7")
                    , pl.col("DM_M28")
                    , pl.col("DM_M42")
                ])
            )

        # Concussion Data Loader
        elif analysis == 'concussion':
            df = (
                role_data
                .join(
                    punt_data
                    , left_on="GSISID"
                    , right_on="GSISID"
                    , how="left"
                    , suffix="_punt"
                )
                .join(
                    play_information
                    , left_on=["GameKey", "PlayID"]
                    , right_on=["GameKey", "PlayID"]
                    , how="left"
                    , suffix="_play"
                )
                .join(
                    game_data
                    , left_on="GameKey"
                    , right_on="GameKey"
                    , how="left"
                    , suffix="_game"
                )
                .join(
                    video_review
                    , left_on=["GameKey", "PlayID", "GSISID"]
                    , right_on=["GameKey", "PlayID", "GSISID"]
                    , how="left"
                    , suffix="_video"
                )
                .with_columns([
                    pl.concat_str([
                        pl.col("GSISID").cast(pl.Utf8)
                        , pl.lit("-")
                        , pl.col("GameKey").cast(pl.Utf8)
                        , pl.lit("-")
                        , pl.col("PlayID").cast(pl.Utf8)
                    ]).alias("PlayKey"),
                    pl.concat_str([
                        pl.col("Primary_Partner_GSISID").cast(pl.Utf8)
                        , pl.lit("-")
                        , pl.col("GameKey").cast(pl.Utf8)
                        , pl.lit("-")
                        , pl.col("PlayID").cast(pl.Utf8)
                    ]).alias("OpponentKey")
                ])
                .select([
                    "PlayKey"
                    , "GSISID"
                    , "GameKey"
                    , "PlayID"
                    , "Position"
                    , "Role"
                    , "Game_Date"
                    , "YardLine"
                    , "Quarter"
                    , "Play_Type"
                    , "Poss_Team"
                    , "Score_Home_Visiting"
                    , "Game_Site"
                    , "Start_Time"
                    , "HomeTeamCode"
                    , "VisitTeamCode"
                    , "StadiumType"
                    , "Turf"
                    , "GameWeather"
                    , "Temperature"
                    , "Player_Activity_Derived"
                    , "Primary_Impact_Type"
                    , "Primary_Partner_Activity_Derived"
                    , "Primary_Partner_GSISID"
                    , 
                ])
                .unique()
            )



        print(f"Tables are holding hands. How cute.")
        return df
    
    except Exception as e: 
        print(f"An error occurred while processing the '{analysis}' analysis: {e}.")
        return None

In [3]:
table_joiner('injury').head()

Tables are holding hands. How cute.


PlayKey,Position,StadiumType,FieldType,Temperature,Weather,PlayType,BodyPart,DM_M1,DM_M7,DM_M28,DM_M42
str,str,str,str,i64,str,str,str,i64,i64,i64,i64
"""26624-1-1""","""QB""","""Outdoor""","""Synthetic""",63,"""Clear and warm""","""Pass""",null,null,null,null,null
"""26624-1-2""","""QB""","""Outdoor""","""Synthetic""",63,"""Clear and warm""","""Pass""",null,null,null,null,null
"""26624-1-3""","""QB""","""Outdoor""","""Synthetic""",63,"""Clear and warm""","""Rush""",null,null,null,null,null
"""26624-1-4""","""QB""","""Outdoor""","""Synthetic""",63,"""Clear and warm""","""Rush""",null,null,null,null,null
"""26624-1-5""","""QB""","""Outdoor""","""Synthetic""",63,"""Clear and warm""","""Pass""",null,null,null,null,null


In [20]:
quals.filter(pl.col("DM_M28").is_not_null()).head()

PlayKey,Position,StadiumType,FieldType,Temperature,Weather,PlayType,BodyPart,DM_M1,DM_M7,DM_M28,DM_M42
str,str,str,str,i64,str,str,str,i64,i64,i64,i64
"""31070-3-7""","""RB""","""Outdoor""","""Natural""",89,"""Sunny""","""Rush""","""Knee""",1,1,1,1
"""33337-8-15""","""TE""","""Outdoors""","""Natural""",81,"""Partly Cloudy""","""Pass""","""Foot""",1,1,1,0
"""33474-19-7""","""FS""","""Outdoor""","""Natural""",79,"""Partly Cloudy""","""Rush""","""Knee""",1,1,1,0
"""34347-5-9""","""WR""","""Oudoor""","""Natural""",48,"""Partly Cloudy""","""Kickoff""","""Knee""",1,1,0,0
"""35570-15-35""","""T""","""Outdoor""","""Natural""",79,"""Clear""","""Pass""","""Knee""",1,1,0,0


In [21]:
quals_with_new_columns = quals.with_columns(
    [ pl.when(pl.col("DM_M1") == 1).then(1).otherwise(0).alias("IsInjured")
    , pl.when(pl.col("DM_M28") == 1).then(1).otherwise(0).alias("IsSevere")
    ]
)

In [23]:
quals_with_new_columns.filter(pl.col("DM_M28").is_not_null()).head().head()

PlayKey,Position,StadiumType,FieldType,Temperature,Weather,PlayType,BodyPart,DM_M1,DM_M7,DM_M28,DM_M42,IsInjured,IsSevere
str,str,str,str,i64,str,str,str,i64,i64,i64,i64,i32,i32
"""31070-3-7""","""RB""","""Outdoor""","""Natural""",89,"""Sunny""","""Rush""","""Knee""",1,1,1,1,1,1
"""33337-8-15""","""TE""","""Outdoors""","""Natural""",81,"""Partly Cloudy""","""Pass""","""Foot""",1,1,1,0,1,1
"""33474-19-7""","""FS""","""Outdoor""","""Natural""",79,"""Partly Cloudy""","""Rush""","""Knee""",1,1,1,0,1,1
"""34347-5-9""","""WR""","""Oudoor""","""Natural""",48,"""Partly Cloudy""","""Kickoff""","""Knee""",1,1,0,0,1,0
"""35570-15-35""","""T""","""Outdoor""","""Natural""",79,"""Clear""","""Pass""","""Knee""",1,1,0,0,1,0


## Clean the Data with the cleaning functions. 

In [63]:
##### Primary Cleaning Functions #####
def clean_injuries():
    """
    Applies data cleaning to surface injury data and writes to 'qualitative' table in db 
    """
    database = "nfl_surface"
    df = table_joiner() 
    df = injury_interpolator(df)
    df = stadium_cleaner(df, df_name='quals')
    df = weather_cleaner(df)
    df = injury_cleaner(df)
    csv_writer(df, "qualitative_injuries")
    del df
    print('Injuries have been cleaned and dressed.')
    # return df

In [42]:
def injury_interpolator(df): 
    """
    Creates two new columns, IsInjured and IsSevere, where any injury sets IsInjured to 1, and any injury over 28 days provides a 1 in IsSevere
    """
    import polars as pl

    df = df.with_columns([ 
        pl.when(pl.col("DM_M1") == 1).then(1).otherwise(0).alias("IsInjured")
        , pl.when(pl.col("DM_M28") == 1).then(1).otherwise(0).alias("IsSevere")
        ])
    print("Injury columns have been added.")
    return df
    

In [64]:
def csv_writer(df, new_file_name):
    """
    Write table to local file as temporary until all cleaning and transformation is done.
    """
    import polars as pl # type: ignore
    import os
       
    path = 'F:/Data/Clean_Data'
    full_path = f"{path}/{new_file_name}.csv"
    
    # Check if file exists
    if os.path.exists(full_path):
        os.remove(full_path)
    
    # Write new file
    df.write_csv(full_path)
    print(f"New file has been written to {full_path}")

In [36]:
def stadium_cleaner(df):
    """
    Noramlizes all stadium types to be either indoor or outdoor per game records. Some of the dome 
    stadiums were listed as open or closed for different games, and these were accounted for.
    All games with dates were checked to ensure null values were indeed outdoor games. 
    """
    import polars as pl  # type: ignore

    stadium_dict = {
        'Outdoor': 'Outdoor'
        , 'Indoors': 'Indoor'
        , 'Oudoor': 'Outdoor'
        , 'Outdoors': 'Outdoor'
        , 'Open': 'Outdoor'
        , 'Closed Dome': 'Indoor'
        , 'Domed, closed': 'Indoor'
        , 'Dome': 'Indoor'
        , 'Indoor': 'Indoor'
        , 'Domed': 'Indoor'
        , 'Retr. Roof-Closed': 'Indoor'
        , 'Outdoor Retr Roof-Open': 'Outdoor'
        , 'Retractable Roof': 'Indoor'
        , 'Ourdoor': 'Outdoor'
        , 'Indoor, Roof Closed': 'Indoor'
        , 'Retr. Roof - Closed': 'Indoor'
        , 'Bowl': 'Outdoor'
        , 'Outddors': 'Outdoor'
        , 'Retr. Roof-Open': 'Outdoor'
        , 'Dome, closed': 'Indoor'
        , 'Indoor, Open Roof': 'Outdoor'
        , 'Domed, Open': 'Outdoor'
        , 'Domed, open': 'Outdoor'
        , 'Heinz Field': 'Outdoor'
        , 'Cloudy': 'Outdoor'
        , 'Retr. Roof - Open': 'Outdoor'
        , 'Retr. Roof Closed': 'Indoor'
        , 'Outdor': 'Outdoor'
        , 'Outside': 'Outdoor'
        , 'outdoor': 'Outdoor'
        , 'Outdoors ': 'Outdoor'
        , 'Indoor, non-retractable roof': 'Indoor'
        , 'Retr. roof - closed': 'Indoor'
        , 'Indoor, fixed roof ': 'Indoor'
        , 'Indoor, Non-Retractable Dome': 'Indoor'
        , 'Indoor, Fixed Roof': 'Indoor'
        , 'Indoor, fixed roof': 'Indoor'
        , 'Indoors (Domed)': 'Indoor'
        , None: 'Outdoor'
        }


    df = df.with_columns(pl.col("StadiumType").replace(stadium_dict)) # This uses the dict to assign naming conventions

    print(f"Someone managed to clean up those stadiums!")
    return df


In [51]:
def weather_cleaner(df):
     """
     Uses mapping to limit the number of different weather groupings. 
     """
     import polars as pl # type: ignore
 
     # If using the concussion dataset, rename the GameWeather column to Weather
     if "GameWeather" in df.columns:
       df = df.rename({"GameWeather": "Weather"})
   
     
     weather_dict = {
            'Clear and warm': 'Clear'
            , 'Mostly Cloudy': 'Cloudy'
            , 'Sunny': 'Clear'
            , 'Clear': 'Clear'
            , 'Cloudy': 'Cloudy'
            , 'Cloudy, fog started developing in 2nd quarter': 'Hazy/Fog'
            , 'Rain': 'Rain'
            , 'Partly Cloudy': 'Cloudy'
            , 'Mostly cloudy': 'Cloudy'
            , 'Cloudy and cold': 'Cloudy'
            , 'Cloudy and Cool': 'Cloudy'
            , 'Rain Chance 40%': 'Rain'
            , 'Controlled Climate': 'Indoor'
            , 'Sunny and warm': 'Clear'
            , 'Partly cloudy': 'Cloudy'
            , 'Clear and Cool': 'Cloudy'
            , 'Clear and cold': 'Cloudy'
            , 'Sunny and cold': 'Clear'
            , 'Indoor': 'Indoor'
            , 'Partly Sunny': 'Clear'
            , 'N/A (Indoors)': 'Indoor'
            , 'Mostly Sunny': 'Clear'
            , 'Indoors': 'Indoor'
            , 'Clear Skies': 'Clear'
            , 'Partly sunny': 'Clear'
            , 'Showers': 'Rain'
            , 'N/A Indoor': 'Indoor'
            , 'Sunny and clear': 'Clear'
            , 'Snow': 'Snow'
            , 'Scattered Showers': 'Rain'
            , 'Party Cloudy': 'Cloudy'
            , 'Clear skies': 'Clear'
            , 'Rain likely, temps in low 40s.': 'Rain'
            , 'Hazy': 'Hazy/Fog'
            , 'Partly Clouidy': 'Cloudy'
            , 'Sunny Skies': 'Clear'
            , 'Overcast': 'Cloudy'
            , 'Cloudy, 50% change of rain': 'Cloudy'
            , 'Fair': 'Clear'
            , 'Light Rain': 'Rain'
            , 'Partly clear': 'Clear'
            , 'Mostly Coudy': 'Cloudy'
            , '10% Chance of Rain': 'Cloudy'
            , 'Cloudy, chance of rain': 'Cloudy'
            , 'Heat Index 95': 'Clear'
            , 'Sunny, highs to upper 80s': 'Clear'
            , 'Sun & clouds': 'Cloudy'
            , 'Heavy lake effect snow': 'Snow'
            , 'Mostly sunny': 'Clear'
            , 'Cloudy, Rain': 'Rain'
            , 'Sunny, Windy': 'Windy'
            , 'Mostly Sunny Skies': 'Clear'
            , 'Rainy': 'Rain'
            , '30% Chance of Rain': 'Rain'
            , 'Cloudy, light snow accumulating 1-3"': 'Snow'
            , 'cloudy': 'Cloudy'
            , 'Clear and Sunny': 'Clear'
            , 'Coudy': 'Cloudy'
            , 'Clear and sunny': 'Clear'
            , 'Clear to Partly Cloudy': 'Clear'
            , 'Cloudy with periods of rain, thunder possible. Winds shifting to WNW, 10-20 mph.': 'Windy'
            , 'Rain shower': 'Rain'
            , 'Cold': 'Clear'
            , 'Partly cloudy, lows to upper 50s.': 'Cloudy'
            , 'Scattered thunderstorms': 'Rain'
            , 'CLEAR': 'Clear'
            , 'Partly CLoudy': 'Cloudy'
            , 'Chance of Showers': 'Rain'
            , 'Snow showers': 'Snow'
            , 'Clear and Cold': 'Clear'
            , 'Cloudy with rain': 'Rain'
            , 'Sunny intervals': 'Clear'
            , 'Clear and cool': 'Clear'
            , 'Cloudy, Humid, Chance of Rain': 'Rain'
            , 'Cloudy and Cold': 'Cloudy'
            , 'Cloudy with patches of fog': 'Hazy/Fog'
            , 'Controlled': 'Indoor'
            , 'Sunny and Clear': 'Clear'
            , 'Cloudy with Possible Stray Showers/Thundershowers': 'Rain'
            , 'Suny': 'Clear'
            , 'T-Storms': 'Rain'
            , 'Sunny and cool': 'Clear'
            , 'Cloudy, steady temps': 'Cloudy'
            , 'Hazy, hot and humid': 'Hazy/Fog'
            , 'Sunny Intervals': 'Clear'
            , 'Partly Cloudy, Chance of Rain 80%': 'Rain'
            , 'Mostly Clear. Gusting ot 14.': 'Windy'
            , 'Mostly CLoudy': 'Cloudy'
            , 'Snow Showers, 3 to 5 inches expected.': 'Snow'
            }



     df = df.with_columns(pl.col("Weather").replace(weather_dict)) # Standardizes the weather to a few main types

     df = df.with_columns(             # Null handling - all null weather conditions for indoor stadiums are filled "indoor"
                pl.when(pl.col("StadiumType") == "Indoor")
                .then(pl.col("Weather").fill_null("Indoor"))
                .otherwise(pl.col("Weather"))
                .alias("Weather")
                )
     
     # For the non-indoor games with null values for weather, to maintain the percentage of games that were clear/cloudy, temperature was used as a divider, above and below 70 degrees
     df = df.with_columns(
                pl.when(pl.col("Temperature") > 70)
                .then(pl.col("Weather").fill_null("Clear"))
                .otherwise(pl.col("Weather"))
                .alias("Weather")
                )
     df = df.with_columns(pl.col("Weather").fill_null("Cloudy"))

     print(f"Looks like the weather has been cleared up.")
     return df


In [55]:
def injury_cleaner(quals):
    """
    Specific to the surface injury data, this filters null values from important data and
    fills nulls that can be appropriately filled. 
    """
    import polars as pl # type: ignore
    quals = quals.filter(pl.col('PlayType').is_not_null()) # 0.14% of rows did not have a play type, and ALL of these were non-injury plays, so they were removed

    quals = quals.with_columns(pl.col("BodyPart").fill_null("No_Injury")) # This fills all null from the join with No Injury

    quals = quals.with_columns(
    pl.col(["DM_M1", "DM_M7", "DM_M28", "DM_M42"]).fill_null(0)) # This fills the nulls from the Join with 0s, since there were no injuries.

    print(f"The injuries have been sanitized.")
    return quals


In [43]:
quals = table_joiner()

Tables are holding hands. How cute.


In [45]:
quals = injury_interpolator(quals)

Injury columns have been added.


In [48]:
quals = stadium_cleaner(quals, 'quals')

Someone managed to clean up those stadiums!


In [52]:
quals = weather_cleaner(quals)

Looks like the weather has been cleared up.


In [56]:
quals = injury_cleaner(quals)

The injuries have been sanitized.


In [57]:
quals.head()

PlayKey,Position,StadiumType,FieldType,Temperature,Weather,PlayType,BodyPart,DM_M1,DM_M7,DM_M28,DM_M42,IsInjured,IsSevere
str,str,str,str,i64,str,str,str,i64,i64,i64,i64,i32,i32
"""26624-1-1""","""QB""","""Outdoor""","""Synthetic""",63,"""Clear""","""Pass""","""No_Injury""",0,0,0,0,0,0
"""26624-1-2""","""QB""","""Outdoor""","""Synthetic""",63,"""Clear""","""Pass""","""No_Injury""",0,0,0,0,0,0
"""26624-1-3""","""QB""","""Outdoor""","""Synthetic""",63,"""Clear""","""Rush""","""No_Injury""",0,0,0,0,0,0
"""26624-1-4""","""QB""","""Outdoor""","""Synthetic""",63,"""Clear""","""Rush""","""No_Injury""",0,0,0,0,0,0
"""26624-1-5""","""QB""","""Outdoor""","""Synthetic""",63,"""Clear""","""Pass""","""No_Injury""",0,0,0,0,0,0


In [58]:
del quals

In [59]:
quals = clean_injuries()

Tables are holding hands. How cute.
Injury columns have been added.
Someone managed to clean up those stadiums!
Looks like the weather has been cleared up.
The injuries have been sanitized.
Injuries have been cleaned and dressed.


In [60]:
quals.head()

PlayKey,Position,StadiumType,FieldType,Temperature,Weather,PlayType,BodyPart,DM_M1,DM_M7,DM_M28,DM_M42,IsInjured,IsSevere
str,str,str,str,i64,str,str,str,i64,i64,i64,i64,i32,i32
"""26624-1-1""","""QB""","""Outdoor""","""Synthetic""",63,"""Clear""","""Pass""","""No_Injury""",0,0,0,0,0,0
"""26624-1-2""","""QB""","""Outdoor""","""Synthetic""",63,"""Clear""","""Pass""","""No_Injury""",0,0,0,0,0,0
"""26624-1-3""","""QB""","""Outdoor""","""Synthetic""",63,"""Clear""","""Rush""","""No_Injury""",0,0,0,0,0,0
"""26624-1-4""","""QB""","""Outdoor""","""Synthetic""",63,"""Clear""","""Rush""","""No_Injury""",0,0,0,0,0,0
"""26624-1-5""","""QB""","""Outdoor""","""Synthetic""",63,"""Clear""","""Pass""","""No_Injury""",0,0,0,0,0,0


In [61]:
del quals

In [65]:
clean_injuries()

Tables are holding hands. How cute.
Injury columns have been added.
Someone managed to clean up those stadiums!
Looks like the weather has been cleared up.
The injuries have been sanitized.
New file has been written to F:/Data/Clean_Data/qualitative_injuries.csv
Injuries have been cleaned and dressed.
